In [1]:
import os
import spacy
import io
from spacy.matcher import Matcher
import pandas as pd
import re
from nltk.corpus import stopwords
import datetime
from datetime import datetime
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import subprocess
import constants as cs
import docxpy
import glob
import numpy as np
# load pre-trained model
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install constants

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install docxpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install docx2txt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import docx2txt

In [8]:
%run constants.py

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
cs

<module 'constants' from '/content/constants.py'>

In [10]:


# FOR INDIAN RESUME RUN THE BELOW FUNCTION TO EXTRACT MOBILE NUMBER
def extract_mobile_number(text):
    phone= re.findall(r'[8-9]{1}[0-9]{9}',text)
    
    if len(phone) > 10:
        return '+' + phone
    else:
        return phone

def extract_email(text):
        email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", text)
        if email:
            try:
                return email[0].split()[0].strip(';')
            except IndexError:
                return None

# Function to remove punctuation and tokenize the text
def tokenText(extText):
   
    # Remove punctuation marks
    punc = '''!()-[]{};:'"\,.<>/?@#$%^&*_~'''
    for ele in extText:
        if ele in punc:
            puncText = extText.replace(ele, "")
            
    # Tokenize the text and remove stop words
    stop_words = set(stopwords.words('english'))
    puncText.split()
    word_tokens = word_tokenize(puncText)
    TokenizedText = [w for w in word_tokens if not w.lower() in stop_words]
    TokenizedText = []
  
    for w in word_tokens:
        if w not in stop_words:
            TokenizedText.append(w)
    return(TokenizedText)            

# Function to extract Name and contact details
def extract_name(Text):
    name = ''  
    for i in range(0,3):
        name = " ".join([name, Text[i]])
    return(name)

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = ['BE','B.E.', 'B.E', 'BS','B.S','B.Com','BCA','ME','M.E', 'M.E.', 'M.S','B.com','10','10+2','BTECH', 'B.TECH', 'M.TECH', 'MTECH', 'SSC', 'HSC', 'C.B.S.E','CBSE','ICSE', 'X', 'XII','10th','12th',' 10th',' 12th','Bachelor of Arts in Mathematics','Master of Science in Analytics','Bachelor of Business Administration','Major: Business Management']

def extract_education(text):
    nlp_text = nlp(text)

    # Sentence Tokenizer
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]


    edu = {}
    # Extract education degree
    for index, t in enumerate(nlp_text):
        for tex in t.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex in EDUCATION and tex not in STOPWORDS:
                edu[tex] = t + nlp_text[index + 1]

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

def extract_skills(resume_text):

        nlp_text = nlp(resume_text)
        noun_chunks = nlp_text.noun_chunks

        # removing stop words and implementing word tokenization
        tokens = [token.text for token in nlp_text if not token.is_stop]
        
        # reading the csv file
        data = pd.read_csv("skills.csv") 
        
        # extract values
        skills = list(data.columns.values)
        
        skillset = []
        
        # check for one-grams (example: python)
        for token in tokens:
            if token.lower() in skills:
                skillset.append(token)
        
        # check for bi-grams and tri-grams (example: machine learning)
        for token in noun_chunks:
            token = token.text.lower().strip()
            if token in skills:
                skillset.append(token)
        
        return [i.capitalize() for i in set([i.lower() for i in skillset])]



def string_found(string1, string2):
        if re.search(r"\b" + re.escape(string1) + r"\b", string2):
            return True
        return False

def extract_entity_sections_grad(text):
    '''
    Helper function to extract all the raw text from sections of resume specifically for 
    graduates and undergraduates
    :param text: Raw text of resume
    :return: dictionary of entities
    '''
    text_split = [i.strip() for i in text.split('\n')]
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(cs.RESUME_SECTIONS_GRAD)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in cs.RESUME_SECTIONS_GRAD:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
    return entities 

# Function to extract experience details
def expDetails(Text):
    global sent
   
    Text = Text.split()
   
    for i in range(len(Text)-2):
        Text[i].lower()
        
        if Text[i] ==  'years':
            sent =  Text[i-2] + ' ' + Text[i-1] +' ' + Text[i] +' '+ Text[i+1] +' ' + Text[i+2]
            l = re.findall('\d*\.?\d+',sent)
            for i in l:
                a = float(i)
            return(a)
            return (sent)

def extract_experience(resume_text):
    '''
    Helper function to extract experience from resume text
    :param resume_text: Plain resume text
    :return: list of experience
    '''
    wordnet_lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # word tokenization 
    word_tokens = nltk.word_tokenize(resume_text)

    # remove stop words and lemmatize  
    filtered_sentence = [w for w in word_tokens if not w in stop_words and wordnet_lemmatizer.lemmatize(w) not in stop_words] 
    sent = nltk.pos_tag(filtered_sentence)

    # parse regex
    cp = nltk.RegexpParser('P: {<NNP>+}')
    cs = cp.parse(sent)
    
    # for i in cs.subtrees(filter=lambda x: x.label() == 'P'):
    #     print(i)
    
    test = []
    
    for vp in list(cs.subtrees(filter=lambda x: x.label()=='P')):
        test.append(" ".join([i[0] for i in vp.leaves() if len(vp.leaves()) >= 2]))

    # Search the word 'experience' in the chunk and then print out the text after it
    x = [x[x.lower().index('experience') + 10:] for i, x in enumerate(test) if x and 'experience' in x.lower()]
    return x

def string_found(string1, string2):
        if re.search(r"\b" + re.escape(string1) + r"\b", string2):
            return True
        return False

def get_score(_dict):
    _len = len(_dict)
    if _len >= 5:
        return 1
    elif _len < 5 and _len > 2:
        return 0.5
    elif _len  == 1:
        return 0.2
    else:
        return 0

def extract_competencies(text, experience_list):
    '''
    Helper function to extract competencies from resume text
    :param resume_text: Plain resume text
    :return: dictionary of competencies
    '''
    experience_text = ' '.join(experience_list)
    competency_dict = {}
    score = 0

    percentage = (100 // len(cs.COMPETENCIES.keys()))

    for competency in cs.COMPETENCIES.keys():
        matches = {}
        for item in cs.COMPETENCIES[competency]:
            if string_found(item, experience_text):
                if competency not in competency_dict.keys():
                    match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                    if item not in matches.keys():
                        matches[item] = [match.group(0)]
                    else:
                        for i in match.groups():
                            matches[item].append(i)    
                    competency_dict[competency] = matches
                else:
                    match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                    if item not in matches.keys():
                        matches[item] = [match.group(0)]
                    else:
                        for i in match.groups():
                            matches[item].append(i)
                    competency_dict[competency] = matches
                score += get_score(competency_dict[competency]) * percentage
    
    competency_dict['score'] = score 
    list=list(competency_dict.keys())
    return(list)

def extract_competencies_score(text, experience_list):
        '''
        Helper function to extract competencies from resume text
        :param resume_text: Plain resume text
        :return: dictionary of competencies
        '''
        experience_text = ' '.join(experience_list)
        competency_dict = {}
        score = 0

        percentage = (100 // len(cs.COMPETENCIES.keys()))

        for competency in cs.COMPETENCIES.keys():
            matches = {}
            for item in cs.COMPETENCIES[competency]:
                if string_found(item, experience_text):
                    if competency not in competency_dict.keys():
                        match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                        if item not in matches.keys():
                            matches[item] = [match.group(0)]
                        else:
                            for i in match.groups():
                                matches[item].append(i)    
                        competency_dict[competency] = matches
                    else:
                        match = re.search(r'([^.|,]*' + item + '[^.|,]*)', experience_text)
                        if item not in matches.keys():
                            matches[item] = [match.group(0)]
                        else:
                            for i in match.groups():
                                matches[item].append(i)
                        competency_dict[competency] = matches
                    score += get_score(competency_dict[competency]) * percentage
        
        competency_dict['score'] = score 
        return(competency_dict['score'])

def extract_dob(text):
        
    result1=re.findall(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}",text)
    result2=re.findall(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}",text)           
    result3= re.findall(r"[\d]{1,2} [ADFJMNOSadfjmnos]\w* [\d]{4}",text)
    result4=re.findall(r"([\d]{1,2})\.([\d]{1,2})\.([\d]{4})",text)
                
    l=[result1,result2,result3,result4]
    for i in l:
        if i==[]:
            continue
        else:
            return i


def extract_text_from_docx(path):
    '''
    Helper function to extract plain text from .docx files
    :param doc_path: path to .docx file to be extracted
    :return: string of extracted text
    '''
    try:
        temp = docx2txt.process(path)
        return temp
    except KeyError:
        return ' '



In [11]:
def extract_text_from_doc(path):
    '''
    Helper function to extract plain text from .docx files
    :param doc_path: path to .docx file to be extracted
    :return: string of extracted text
    '''
    try:
        temp = doc2txt.process(path)
        return temp
    except KeyError:
        return ' '

In [12]:
def extract_text_from_pdf(path):
    '''
    Helper function to extract plain text from .docx files
    :param doc_path: path to .docx file to be extracted
    :return: string of extracted text
    '''
    try:
        temp = pdf2txt.process(path)
        return temp
    except KeyError:
        return ' '

In [13]:
df = pd.DataFrame(columns=['Name','Mobile No.', 'Email','DOB','Education Qualifications','Skills','Experience (Years)','Last Position','Competence','competence score'], dtype=object)

In [14]:
i=0
path_input = r"/content/Peoplesoft Admin_AnubhavSingh.docx"
if path_input.endswith('.docx'):
    text = extract_text_from_docx(path_input)
    tokText = tokenText(text)
    df.loc[i,'Name']=extract_name(tokText)
    df.loc[i,'Mobile No.']=extract_mobile_number(text)
    df.loc[i,'Email']=extract_email(text)
    df.loc[i,'DOB']=extract_dob(text)
    df.loc[i,'Education Qualifications']=extract_education(text)
    df.loc[i,'Skills']=extract_skills(text)
    df.loc[i,'Experience (Years)']=expDetails(text) 
    experience_list1=extract_entity_sections_grad(text) 

    if 'experience' in experience_list1:
        i=0
        experience_list=experience_list1['experience']
        df.loc[i,'Last Position']=extract_experience(text)
        df.loc[i,'competence score']=extract_competencies_score(text,experience_list)

    else:
        df.loc[i,'Last Position']='NA'
        df.loc[i,'Competence']='NA'
        df.loc[i,'competence score']='NA'
    i+=1

In [15]:


df



,Name,Mobile No.,Email,DOB,Education Qualifications,Skills,Experience (Years),Last Position,Competence,competence score
0,Anubhav Kumar Singh,[],None,[11/09/1990],[],"[Certification, Github, Migration, Unix, Shell...",None,"[, Installing Oracle Policy Automation]",NaN,8.0


In [16]:
pip install textract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pdfminer.six-20191110-py2.py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20221105
    Uninstalling pdfminer.six-20221105:
      Successfully uninstalled pdfminer.six-20221105
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.9.0 requires pdfminer.six==20221105, but you have pdfminer-six 20191110 which is incompatible.


In [17]:
pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20191110
    Uninstalling pdfminer.six-20191110:
      Successfully uninstalled pdfminer.six-20191110
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textract 1.6.5 requires pdfminer.six==20191110, but you have pdfminer-six 20221105 which is incompatible.


In [18]:
from docx import Document

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    paragraphs = [p.text for p in doc.paragraphs]
    return '\n'.join(paragraphs)

In [19]:
import pdfplumber

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
        return text

# Rest of the code remains the same


In [20]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import PyPDF2

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        num_pages = reader.numPages
        text = ''
        for page in range(num_pages):
            text += reader.getPage(page).extractText()
        return text

In [22]:
import os
from docx import Document

def extract_text_from_docx(file_path):
    document = Document(file_path)
    paragraphs = [p.text for p in document.paragraphs]
    return '\n'.join(paragraphs)

def extract_text_from_pdf(file_path):
    # Code to extract text from PDF goes here
    # Replace this with your own implementation or use a library like PyPDF2 or pdfminer


    df = pd.DataFrame(columns=['Name', 'Mobile No.', 'Email', 'DOB', 'Education Qualifications', 'Skills',
                               'Experience (Years)', 'Last Position', 'Competence', 'competence score'])

In [23]:
pip install docx2txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import docx2txt

In [25]:
from docx import Document

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    paragraphs = [p.text for p in doc.paragraphs]
    return "\n".join(paragraphs)


In [26]:
import subprocess

def extract_text_from_doc(file_path):
    process = subprocess.Popen(['antiword', file_path], stdout=subprocess.PIPE)
    output, _ = process.communicate()
    return output.decode('utf-8')


In [27]:
import PyPDF2

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text


In [28]:
import docx2txt

def extract_text_from_doc(file_path):
    text = docx2txt.process(file_path)
    return text

In [29]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
!pdfplumber

Traceback (most recent call last):
  File "/usr/local/bin/pdfplumber", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pdfplumber/cli.py", line 61, in main
    with PDF.open(args.infile, pages=args.pages, laparams=args.laparams) as pdf:
  File "/usr/local/lib/python3.10/dist-packages/pdfplumber/pdf.py", line 78, in open
    return cls(
  File "/usr/local/lib/python3.10/dist-packages/pdfplumber/pdf.py", line 40, in __init__
    self.doc = PDFDocument(PDFParser(stream), password=password)
  File "/usr/local/lib/python3.10/dist-packages/pdfminer/pdfparser.py", line 46, in __init__
    PSStackParser.__init__(self, fp)
  File "/usr/local/lib/python3.10/dist-packages/pdfminer/psparser.py", line 545, in __init__
    PSBaseParser.__init__(self, fp)
  File "/usr/local/lib/python3.10/dist-packages/pdfminer/psparser.py", line 193, in __init__
    self.seek(0)
  File "/usr/local/lib/python3.10/dist-packages/pdfminer/psparser.py", line 557, in seek
    PSBas

In [32]:
import os
import glob
import pandas as pd
import docx

# Folder path containing the files
folder_path = r"/content/drive/MyDrive/Resumes"

# File extensions to consider
file_extensions = ['pdf', 'docx', 'doc']

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Name', 'Mobile No.', 'Email', 'DOB', 'Education Qualifications',
                           'Skills', 'Experience (Years)', 'Last Position', 'Competence',
                           'competence score'])

# Retrieve file paths matching the extensions
file_paths = []
for ext in file_extensions:
    file_paths.extend(glob.glob(os.path.join(folder_path, f"*.{ext}")))

# Process each file
for i, path_input in enumerate(file_paths):
    try:
        if path_input.endswith('.docx'):
            # Process DOCX files
            doc = docx.Document(path_input)
            paragraphs = [p.text for p in doc.paragraphs]
            text = "\n".join(paragraphs)
        elif path_input.endswith('.doc'):
            # Process DOC files
            doc = docx.Document(path_input)
            paragraphs = [p.text for p in doc.paragraphs]
            text = "\n".join(paragraphs)
        elif path_input.endswith('.pdf'):
            # Process PDF files
            text = extract_text_from_pdf(path_input)
        else:
            # Skip unsupported file formats
            continue

        tokText = tokenText(text)
        df.loc[i, 'Name'] = extract_name(tokText)
        df.loc[i, 'Mobile No.'] = extract_mobile_number(text)
        df.loc[i, 'Email'] = extract_email(text)
        df.loc[i, 'DOB'] = extract_dob(text)
        df.loc[i, 'Education Qualifications'] = extract_education(text)
        df.loc[i, 'Skills'] = extract_skills(text)
        df.loc[i, 'Experience (Years)'] = expDetails(text)
        experience_list1 = extract_entity_sections_grad(text)

        if 'experience' in experience_list1:
            experience_list = experience_list1['experience']
            df.loc[i, 'Last Position'] = extract_experience(text)
           
            df.loc[i, 'competence score'] = extract_competencies_score(text, experience_list)
        else:
            df.loc[i, 'Last Position'] = 'NA'
            df.loc[i, 'Competence'] = 'NA'
            df.loc[i, 'competence score'] = 'NA'

    except ValueError as e:
        if "file is not a Word file" in str(e):
            # Skip files that are not valid Word files
            print(f"Skipped file: {path_input} - Not a valid Word file")
        else:
            # Handle other ValueErrors
            print(f"Error processing file: {path_input} - {str(e)}")

# Print the resulting DataFrame
print(df)


Error processing file: /content/drive/MyDrive/Resumes/React Developer_PavasGoswami.doc - file '/content/drive/MyDrive/Resumes/React Developer_PavasGoswami.doc' is not a Word file, content type is 'application/vnd.openxmlformats-officedocument.themeManager+xml'
Error processing file: /content/drive/MyDrive/Resumes/React Developer_Vinay Reddy.doc - file '/content/drive/MyDrive/Resumes/React Developer_Vinay Reddy.doc' is not a Word file, content type is 'application/vnd.openxmlformats-officedocument.themeManager+xml'
Error processing file: /content/drive/MyDrive/Resumes/React JS Developer_AnjaniPriyadarshini.doc - file '/content/drive/MyDrive/Resumes/React JS Developer_AnjaniPriyadarshini.doc' is not a Word file, content type is 'application/vnd.openxmlformats-officedocument.themeManager+xml'
                                  Name    Mobile No.  \
0                             Page | 1            []   
1                        Name Ravali P            []   
2                  SUSOVAN BAG 

In [33]:
# Assuming the data is stored in a variable called 'df'
data_str = df.to_string(index=False)

# Print the data string
print(data_str)

                              Name   Mobile No.                     Email          DOB Education Qualifications                                                                                                                                                                                                                                                           Skills Experience (Years)                                                     Last Position Competence competence score
                          Page | 1           []                      None [23/01/1995]                       []                                                                                                                              [Design, Queries, Aws, Js, Analytical, Unix, Ui, Api, Android, Pattern, Xml, Windows, Css, System, Mobile, Programming, Html, Java]                2.4                                                                []        NaN              4.0
                     Name Ravali P  

In [34]:
# Save DataFrame to CSV file
output_file = '/content/drive/MyDrive/Resumes.text/resumes_data.csv'
df.to_csv(output_file, index=False)

print("CSV file saved successfully.")

CSV file saved successfully.


In [35]:
resume=pd.read_csv="/content/drive/MyDrive/Resumes.text/resumes_data.csv"